# Creating Venue Table

In [0]:
import pyspark.sql.functions as F

### Reading in the data

In [0]:
# replace with reading parquet files in the future
df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.*.json.gz')

In [0]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _id: string (nullable = true)
 |    |    |-- avatar: string (nullable = true)
 |    |    |-- bio: string (nullable = true)
 |    |    |-- email: string (nullable = true)
 |    |    |-- gid: string (nullable = true)
 |    |    |-- homepage: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- name_zh: string (nullable = true)
 |    |    |-- oid: string (nullable = true)
 |    |    |-- oid_zh: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |    |    |-- org: string (nullable = true)
 |    |    |-- org_zh: string (nullable = true)
 |    |    |-- orgid: string (nullable = true)
 |    |    |-- orgs: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- orgs_zh: array (nullable = true)
 |    |    |    |-- element:

### Preprocessing
Replacing venues with empty fileds with null.

In [0]:
venue_is_empty = (
    F.col("venue.issn").isNull() &
    F.col("venue.name").isNull() &
    F.col("venue.name_d").isNull() &
    F.col("venue.name_s").isNull() &
    F.col("venue.online_issn").isNull() &
    F.col("venue.publisher").isNull() &
    F.col("venue.raw").isNull() &
    F.col("venue.raw_zh").isNull() &
    F.col("venue.t").isNull()
)
df_unified_none = df.withColumn("venue", F.when(venue_is_empty, None).otherwise(F.col("venue")))

### Creating new ids for venues.

We use issn when _id is null. If both are null we ignore it. We lose venues for <5% of rows.

In [0]:
df_updated_ids = df_unified_none.withColumn("venue", F.col("venue").withField("_id", F.coalesce(F.col("venue._id"), F.col("venue.issn"))))

### Creating the venues table

Creating the venues table with venues that have ids.

Dropping useless columns: src, type, sid. Column t gets renamed into type, since it has useful information: J = Journal, C = Workshop or Conference

In [0]:
raw_venues_df = df_updated_ids.select("venue.*").filter(F.col("_id").isNotNull()).withColumn("type", F.col("t")).drop("src", "t", "sid").distinct()
display(raw_venues_df.limit(10))

_id,issn,name,name_d,name_s,online_issn,publisher,raw,raw_zh,sid,type
1877-0509,1877-0509,null,null,null,null,Elsevier,Procedia Computer Science,null,procedia-computer-science,J
5390885520f70186a0d8e8b8,null,null,null,null,null,null,Games,null,null,null
1053-8119,1053-8119,null,null,null,null,Academic Press,NeuroImage,null,neuroimage,J
0165-0114,0165-0114,null,null,null,null,North-Holland,Fuzzy Sets and Systems,null,fuzzy-sets-and-systems,J
53a7258b20f7420be8b51bd6,null,null,null,null,null,null,ICCS,null,null,null
2169-3536,2169-3536,null,null,null,null,null,IEEE ACCESS,null,IEEE ACCESS,J
0018-9545,0018-9545,null,null,null,1939-9359,null,IEEE Transactions on Vehicular Technology,null,25,J
0097-8493,0097-8493,null,null,null,null,Pergamon,Computers & Graphics,null,computers-and-graphics,J
53a72e1520f7420be8c7e9a4,null,null,null,null,null,null,SDM,null,null,null
555037cd7cea80f95419a56b,null,null,null,null,null,null,PRICAI,null,null,null


### Combining rows with the same id

Some rows with matching id have fields filled and some do not. Make the rows distinct by _id and fill the fields with the first non-null value.

In [0]:
venue_columns = (
    "issn",
    "name",
    "name_d",
    "name_s",
    "raw",
    "raw_zh",
    "online_issn",
    "publisher",
    "type"
)

distinct_raw_venues_df = raw_venues_df.groupBy(F.col("_id")).agg(*(F.first(F.col(col), ignorenulls=True).alias(col) for col in venue_columns))

### Creating a name column

Combining various columns that may contain name of the venue into a single column

In [0]:
venue_df = (
    distinct_raw_venues_df
    .withColumn("name", F.coalesce(
            F.col("name"), 
            F.col("name_d"), 
            F.col("raw"), 
            F.col("name_s"), 
            F.col("raw_zh")
        ))
    .drop("name_d", "raw", "name_s", "raw_zh")    
)

display(venue_df.orderBy(F.rand()).limit(10))

_id,issn,name,online_issn,publisher,type
57d08aa00a3ac5db49921a90,null,GamifIR@ECIR,null,ACM,null
5390a44320f70186a0e70f71,null,ISPASS '08 Proceedings of the ISPASS 2008 - IEEE International Symposium on Performance Analysis of Systems and software,null,null,null
5736ae6bd39c4f40a7978b44,null,OSS4MDE@MoDELS,null,null,null
53a732cf20f7420be8db745d,null,Parallel Computing,null,null,null
53a7289920f7420be8bb2ab9,null,MESOCA,null,,null
53907b7f20f770854f5ed389,null,Predictive data mining: a practical guide,null,null,null
53a7313620f7420be8d2d604,null,Journal of Machine Learning Research,null,null,null
53e18c1f20f7dfbc07e8ffb2,null,Symposium on Applications and the Internet,null,null,null
55f95973c35f4fb0d21ce642,null,Asia and South Pacific Design Automation Conference,null,null,null
0038-9056,0038-9056,STARKE,1521-379X,null,J


In [0]:
venue_df.count()

Out[94]: 50560

### Saving the table

In [0]:
venue_df.write.format("delta").mode("overwrite").saveAsTable("venues") 

### Testing

In [0]:
main_table = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.4.json.gz').withColumn("venue", F.coalesce(F.col("venue._id"), F.col("venue.issn")))
venue_table = spark.table("venues")

In [0]:
display(main_table.join(venue_table, main_table.venue ==  venue_table._id, "leftouter").limit(10))

_id,abstract,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year,_id,issn,name,online_issn,publisher,type
53e9a751b7602d970308796b,"In this paper I investigate the problem of tagging elements of a set, and the elements of those elements, uniquely, when they admit an order, and two boundary elements are tagged. A heuristic sorting algorithm is also investigated. (Updated grammar and spellings.)","List(List(53f431cddabfaec09f14c449, null, null, null, Sayandeep Khan, null, null, null, null, null, null, null, null, null, null))",,"List(Heuristic, Cave, Yet another, Numbering scheme, Treasure, Computer science, Algorithm, Pallet, Theoretical computer science, Sorting problem, Sorting algorithm)",,,,List(),en,0,,,https://static.aminer.cn/storage/pdf/arxiv/12/1206/1206.5335.pdf,null,The McDougal Cave and Counting issues,"List(http://arxiv.org/abs/1206.5335, https://arxiv.org/abs/1206.5335)",555036d37cea80f95415b0ba,abs/1206.5335,2012,555036d37cea80f95415b0ba,null,arXiv: Learning,null,null,null
53e9a751b7602d9703087a5c,,"List(List(53f4e88adabfaeb1a7cd45fb, null, null, null, Wen-mei W. Hwu, null, null, null, null, null, null, null, null, null, null), List(5489313adabfaed7b5fa39db, null, null, null, Thomas M. Conte, null, null, null, null, null, null, null, null, null, null))",,"List(Computer science, Parallel computing, Multiprocessing, Instruction prefetch)",,,,List(),en,0,,,null,null,A Simulation Study of Simultaneous Vector Prefetch Performance in Multiprocessor Memory Subsystems (Extended Abstract),List(),53a72ca420f7420be8c4aef0,,1989,53a72ca420f7420be8c4aef0,null,Measurement and Modeling of Computer Systems,null,,C
53e9a751b7602d9703087aa9,"In order to efficiently encrypt multimedia streams deliv- ered in real-time environments, a Fast Encryption Al- gorithm for Multimedia (FEA-M) was proposed (8, 9). Cryptanalyses of this technique (1, 3, 4, 5) have iden- tified its weaknesses and an improved variant has been suggested in (5). In this paper, we identify further weak- nesses in the original FEA-M and also in the improved variant. Our solution provides message integrity, guaran- tees zero packet loss and protects against specific known plaintext attacks.","List(List(53f4783adabfaee4dc89484a, null, null, null, Depeng Li, null, null, null, null, null, null, null, null, null, null), List(53f4391bdabfaee4dc79cecf, null, null, null, Srinivas Sampalli, null, null, null, null, null, null, null, null, null, null))",,"List(Message integrity, Computer science, Known-plaintext attack, Computer network, Cryptanalysis, Packet loss, Encryption, Probabilistic encryption, Multimedia, Ciphertext-only attack, Plaintext)",,,2,"List(multimedia encryption, cryptanalysis, reliable transportation, real time, packet loss, known plaintext attack)",en,1,192,187,null,"List(53e99a20b7602d9702277bc5, 53e9a501b7602d9702e209aa, 53e9bb29b7602d9704762958, 53e99885b7602d97020bd403, 558a40f1e4b0b32fcb35b90c, 53e99e8cb7602d9702754f1d, 53e99d37b7602d97025ef9ac)",Further Improvements of Fast Encryption Algoirthm for Multimedia,List(http://ijns.femto.com.tw/contents/ijns-v7-n2/ijns-2008-v7-n2-p187-192.pdf),555036e47cea80f954164b11,7,2008,555036e47cea80f954164b11,null,International Journal of Network Security,null,null,null
53e9a751b7602d9703087af4,,"List(List(54303191dabfaeca69bd4998, null, null, null, Ed Dawson, null, null, null, null, null, null, null, null, null, 617262), List(54409306dabfae7d84b84d7f, null, null, null, Duncan S. Wong, null, null, null, null, null, null, null, null, null, 640091))",,null,isbn,null,null,List(),en,25,null,null,,null,"Information Security Practice and Experience, Third International Conference, ISPEC 2007, Hong Kong, China, May 7-9, 2007, Proceedings",null,555037437cea80f95417d301,4464,2007,555037437cea80f95417d301,null,Information Security Practice and Experience,null,null,null
53e9a751b7602d9703087997,"In this paper, we present a new framework for shape modelling and